In [1]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 10, 1, Finished, Available)

In [2]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("TipoViolacao", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 10, 2, Finished, Available)

In [3]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Tipo de Violação.csv')

StatementMeta(tcccibele, 10, 3, Finished, Available)

In [4]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 10, 4, Finished, Available)

In [5]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'TipoViolacao':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 10, 5, Finished, Available)

In [6]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/TipoViolacao/')

StatementMeta(tcccibele, 10, 6, Finished, Available)

In [7]:
df = spark.read.parquet('/history_zone/2019/TipoViolacao/')
df.toPandas()

StatementMeta(tcccibele, 10, 7, Finished, Available)

,TipoViolacao,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Ameaça,371,350,331,269,275,248,263,250,228,277,261,133
1,Cárcere Privado,308,227,203,190,180,135,165,169,143,336,308,147
2,Feminicídio,12,13,6,5,4,2,8,4,4,2,1,0
3,Tentativa de Feminicídio,795,592,407,329,376,256,272,372,247,35,24,10
4,Trabalho Escravo,3,4,4,1,0,2,0,1,1,0,1,0
5,Tráfico de Mulheres,3,1,2,4,5,1,4,8,1,1,4,4
6,Violência no Esporte,0,0,0,0,0,0,0,0,0,0,0,0
7,Violência contra Diversidade Religiosa,1,5,2,0,1,2,1,0,0,2,1,0
8,Violência Doméstica e Familiar (Total),7004,5547,6644,5750,5759,5065,5205,6223,6109,5831,5391,2910
9,Descumprimento de Medidas Protetivas,246,191,219,207,189,156,222,245,273,313,291,174
